# Dataset Distillation Learning

##Loading Packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls /content/drive/MyDrive/ECE1512\ Assignment\ 2/*.py

'/content/drive/MyDrive/ECE1512 Assignment 2/networks.py'
'/content/drive/MyDrive/ECE1512 Assignment 2/utils.py'


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/ECE1512 Assignment 2')

In [4]:
!pip install ptflops
import networks
from utils import get_dataset, get_network, evaluate_synset
import torch
import torch.nn as nn
import numpy as np
from ptflops import get_model_complexity_info
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import os
import time
import copy
import argparse
import numpy as np
import torch
import torch.nn as nn
#from torchvision.utils import save_image, make_grid
#from ptflops import get_model_complexity_info
from utils import get_dataset, get_network, TensorDataset, get_time
import matplotlib.pyplot as plt
builder = tfds.builder('mnist')
NUM_EPOCHS = 20
BATCH_SIZE = 256
#MODEL = 'ConvNet'
#IMAGE_PER_CLASS = 10
#EVAL_MODE = 'SS'
#LR_IMG = 1
#LR_NET = 0.01
NUMBER_CLASSES = 10
#ITERATIONS = 10
#NUMBER_INITIALIZATIONS = 100
#NUMBER_STEP_OPT = 50
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

  Preparing metadata (setup.py) ... done
  Created wheel for ptflops: filename=ptflops-0.7.1.2-py3-none-any.whl size=13213 sha256=ea1f0f33ccc494ffafbbf7e5eddd757e5ce6f28a26785a000e71dbdda16dc791
  Stored in directory: /root/.cache/pip/wheels/9d/90/07/20e8c3221349a85d63b319593e1bcbb6e0c995d2e2bcc5d775
Successfully built ptflops


#Create dataset for MHIST

In [5]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [6]:
###############
#https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/6f5f4fbfa307e6755c583593b9623cfb/custom_dataset_transforms_loader.ipynb#scrollTo=agFaBk4wlbB2
###############
class MHIST(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.mhist_labels = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.mhist_labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.mhist_labels.iloc[idx, 0])
        images = io.imread(img_name)
        #if self.mhist_labels.iloc[idx, 1] == 'HP':
        labels = self.mhist_labels.iloc[idx, 1]
        #labels = np.array([labels])
        #labels = labels.astype('float').reshape(-1, 2)
        #sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            images = self.transform(images)

        return images, labels

In [7]:
BATCH_SIZE = 2
mhist_dataset = MHIST(csv_file='/content/drive/MyDrive/images/annotations_v2.csv',root_dir='/content/drive/MyDrive/images/', transform=transforms.ToTensor())
train_set, test_set = torch.utils.data.random_split(mhist_dataset, [2522, 630])
trainloader_mhist = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
testloader_mhist = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

# 2.a Train the selected model with the original dataset

In [8]:
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset("MNIST", "/content/drive/MyDrive/MNIST")

In [9]:
dst_train

Dataset MNIST
    Number of datapoints: 60000
    Root location: /content/drive/MyDrive/MNIST
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.1307], std=[0.3081])
           )

In [10]:
model = get_network("ConvNet", channel, num_classes, im_size).to(DEVICE)

In [11]:
mnist_train, mnist_test = dst_train, dst_test

In [12]:
opt = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)
lossFn = nn.CrossEntropyLoss()

In [13]:
trainloader = torch.utils.data.DataLoader(dst_train, batch_size=BATCH_SIZE, shuffle=True)

In [14]:
trainloader

In [9]:
def compute_num_correct(model, testloader):
  model.eval()
  valCorrect = 0
  #val_loss = 0
  for (images, labels) in testloader:
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)
    pred = model(images)
    #val_loss += lossFn(pred, labels)
    valCorrect += (pred.argmax(1) == labels).type(torch.float).sum().item()
  return valCorrect

def train_and_evaluate(model, trainloader, testloader):
  test_acc_list = []
  print('%s Training begins'%get_time())
  for epoch in range(0, NUM_EPOCHS+1):
    print('Epoch {}: '.format(epoch), end='')
    #Training
    model.train()
    train_loss = 0.

    for (images, labels) in trainloader:
      #images, labels = data
      images = images.to(DEVICE)
      labels = labels.to(DEVICE)
      opt.zero_grad()
      pred = model(images)

      loss = lossFn(pred, labels)
      loss.backward()
      opt.step()
      #train_loss += loss.item()
    #Evaluation
    test_acc = compute_num_correct(model, testloader)/len(testloader.dataset)
    print("Class_accuracy: " + '{:.2f}%'.format(test_acc * 100))
    test_acc_list.append(test_acc)
###################
#caiquanshi
#https://github.com/caiquanshi/ECE1512_2022F_ProjectRepo_CaiquanShi_JunchengMo/blob/main/Project%20B/task1/projectb_task1.ipynb
###################
  print('%s Training ends'%get_time())
  macs, params = get_model_complexity_info(model, (channel, im_size[0], im_size[1]), as_strings=True,
                                           print_per_layer_stat=False, verbose=True)
  print("Flops: {}".format(macs))

  return test_acc_list

In [ ]:
mnist_acc = train_and_evaluate(model, trainloader, testloader)

In [15]:
##################
#
##################
class Swish(nn.Module): # Swish(x) = x∗σ(x)
    def __init__(self):
        super().__init__()

    def forward(self, input):
        return input * torch.sigmoid(input)

class ConvNet7(nn.Module):
    def __init__(self, channel, num_classes, net_width, net_depth, net_act, net_norm, net_pooling, im_size = (224,224)):
        super(ConvNet7, self).__init__()

        self.features, shape_feat = self._make_layers(channel, net_width, net_depth, net_norm, net_act, net_pooling, im_size)
        num_feat = shape_feat[0]*shape_feat[1]*shape_feat[2]
        self.classifier = nn.Linear(num_feat, num_classes)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def embed(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        return out

    def _get_activation(self, net_act):
        if net_act == 'sigmoid':
            return nn.Sigmoid()
        elif net_act == 'relu':
            return nn.ReLU(inplace=True)
        elif net_act == 'leakyrelu':
            return nn.LeakyReLU(negative_slope=0.01)
        elif net_act == 'swish':
            return Swish()
        else:
            exit('unknown activation function: %s'%net_act)

    def _get_pooling(self, net_pooling):
        if net_pooling == 'maxpooling':
            return nn.MaxPool2d(kernel_size=2, stride=2)
        elif net_pooling == 'avgpooling':
            return nn.AvgPool2d(kernel_size=2, stride=2)
        elif net_pooling == 'none':
            return None
        else:
            exit('unknown net_pooling: %s'%net_pooling)

    def _get_normlayer(self, net_norm, shape_feat):
        # shape_feat = (c*h*w)
        if net_norm == 'batchnorm':
            return nn.BatchNorm2d(shape_feat[0], affine=True)
        elif net_norm == 'layernorm':
            return nn.LayerNorm(shape_feat, elementwise_affine=True)
        elif net_norm == 'instancenorm':
            return nn.GroupNorm(shape_feat[0], shape_feat[0], affine=True)
        elif net_norm == 'groupnorm':
            return nn.GroupNorm(4, shape_feat[0], affine=True)
        elif net_norm == 'none':
            return None
        else:
            exit('unknown net_norm: %s'%net_norm)

    def _make_layers(self, channel, net_width, net_depth, net_norm, net_act, net_pooling, im_size):
        layers = []
        in_channels = channel
        if im_size[0] == 28:
            im_size = (32, 32)
        shape_feat = [in_channels, im_size[0], im_size[1]]
        for d in range(net_depth):
            layers += [nn.Conv2d(in_channels, net_width, kernel_size=3, padding=3 if channel == 1 and d == 0 else 1)]
            shape_feat[0] = net_width
            if net_norm != 'none':
                layers += [self._get_normlayer(net_norm, shape_feat)]
            layers += [self._get_activation(net_act)]
            in_channels = net_width
            if net_pooling != 'none':
                layers += [self._get_pooling(net_pooling)]
                shape_feat[1] //= 2
                shape_feat[2] //= 2

        return nn.Sequential(*layers), shape_feat

In [17]:
def get_default_convnet_setting():
    net_width, net_depth, net_act, net_norm, net_pooling = 128, 3, 'relu', 'instancenorm', 'avgpooling'
    return net_width, net_depth, net_act, net_norm, net_pooling

In [18]:
net_width, net_depth, net_act, net_norm, net_pooling = get_default_convnet_setting()
model_convnet7 = ConvNet7(channel=3, num_classes=2, net_width=net_width, net_depth=7, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling, im_size=(224,224)).to(DEVICE)
opt = torch.optim.SGD(model_convnet7.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)
lossFn = nn.CrossEntropyLoss()
channel = 3
im_size = (224,224)

In [ ]:
mhist_acc = train_and_evaluate(model_convnet7, trainloader_mhist, testloader_mhist)

In [20]:
model = model_convnet7
channel = 3
im_size = (224, 224)
macs, params = get_model_complexity_info(model, (channel, im_size[0], im_size[1]), as_strings=True,
                                           print_per_layer_stat=False, verbose=True)
print("Flops: {}".format(macs))

Flops: 2.7 GMac


#2.b

In [13]:
from torchvision.utils import save_image, make_grid
IMAGE_PER_CLASS = 10
EVAL_MODE = 'SS'
TRAIN_NET_EPOCH = 20
BATCH_SIZE = 256
LR_IMG = 1
LR_NET = 0.01
NUMBER_CLASSES = 10
ITERATIONS = 10
NUMBER_INITIALIZATIONS = 100
NUMBER_STEP_OPT = 50

In [14]:
def evaluate(model,valid_data):
    model.eval()
    total = 0
    correct = 0
    for data in valid_data:
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        output = model(inputs)
        output = torch.argmax(output, dim=1)
        correct += torch.sum(output == labels)
        total += BATCH_SIZE
    return correct / float(total)

def train_and_eval(model, image_syn_eval, syn_lable, dst_test):
  dst_train = TensorDataset(image_syn_eval, syn_lable)
  trainloader = torch.utils.data.DataLoader(dst_train, batch_size=BATCH_SIZE, shuffle=True)
  validLoader = torch.utils.data.DataLoader(dst_test, batch_size=BATCH_SIZE, shuffle=True)
  optimizer = torch.optim.SGD(model.parameters(), lr=LR_NET)
  cee = nn.CrossEntropyLoss()
  valid_accs = []
  for epoch in range(TRAIN_NET_EPOCH):
    model.train()
    train_loss = 0.

    for data in trainloader:
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = cee(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    valid_acc = evaluate(model, validLoader)
    valid_accs.append(valid_acc)
    print("Epoch: {} \tTraining Cost: {:.6f}\t "
        "Valid Acc: {}.".format(epoch, train_loss, valid_acc))
  return valid_accs

def train_model(model, image_syn_eval, syn_lable, epochs):
  dst_train = TensorDataset(image_syn_eval, syn_lable)
  trainloader = torch.utils.data.DataLoader(dst_train, batch_size=BATCH_SIZE, shuffle=True)
  optimizer = torch.optim.SGD(model.parameters(), lr=LR_NET)
  cee = nn.CrossEntropyLoss().to(DEVICE)
  for e in range(epochs):
    model.train()
    train_loss = 0.

    for data in trainloader:
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = cee(outputs, labels)
        loss.backward()
        optimizer.step()

def match_mse_loss(gw_syn, gw_real):
  gw_real_vec = []
  gw_syn_vec = []
  for ig in range(len(gw_real)):
      gw_real_vec.append(gw_real[ig].reshape((-1)))
      gw_syn_vec.append(gw_syn[ig].reshape((-1)))
  gw_real_vec = torch.cat(gw_real_vec, dim=0)
  gw_syn_vec = torch.cat(gw_syn_vec, dim=0)
  dis = torch.sum((gw_syn_vec - gw_real_vec) ** 2)
  return dis


def condensed_images_and_evaluate(dataset_name, model_name, eval_mode_name, randmon_init=False):
  channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(dataset_name, DATAPATH)
  class_index = {}
  data_save = []
  for i in range(len(dst_train)):
    lable = dst_train[i][1]
    if lable in class_index:
      class_index[lable].append(i)
    else:
      class_index[lable] = [i]
  images_all = [torch.unsqueeze(dst_train[i][0], dim=0) for i in range(len(dst_train))]
  images_all = torch.cat(images_all, dim=0).to(DEVICE)
  syn_image = []
  for c in range(num_classes):
    idx_shuffle = np.random.permutation(class_index[c])[:IMAGE_PER_CLASS]
    syn_image.append(images_all[idx_shuffle])
  syn_image = torch.cat(syn_image, dim=0).to(DEVICE)
  syn_image.requires_grad = True
  syn_lable = []
  for c in range(num_classes):
    syn_lable += [c for i in range(IMAGE_PER_CLASS)]
  if randmon_init:
    syn_image = torch.randn(size=(num_classes*IMAGE_PER_CLASS, channel, im_size[0], im_size[1]), dtype=torch.float, requires_grad=True, device=DEVICE)
  syn_lable = torch.tensor(syn_lable,device=DEVICE)
  optimizer_img = torch.optim.SGD([syn_image, ], lr=LR_IMG)
  criterion = nn.CrossEntropyLoss().to(DEVICE)
  total_loss = 0
  print('%s training begins'%get_time())
  for exp in range(NUMBER_INITIALIZATIONS):
    model = get_network(model_name, channel, num_classes, im_size).to(DEVICE)
    model.train()
    net_parameters = list(model.parameters())

    for it in range(ITERATIONS):
      loss = torch.tensor(0.0).to(DEVICE)
      for c in range(num_classes):
        strat_index = c*IMAGE_PER_CLASS
        img_real = images_all[np.random.permutation(class_index[c])[:BATCH_SIZE]]
        lab_real = torch.ones((BATCH_SIZE,), device=DEVICE, dtype=torch.long) * c
        img_syn = syn_image[strat_index: strat_index + IMAGE_PER_CLASS].reshape((IMAGE_PER_CLASS, channel, im_size[0], im_size[1]))
        lab_syn = torch.ones((IMAGE_PER_CLASS,), device=DEVICE, dtype=torch.long) * c
        output_real = model(img_real)
        loss_real = criterion(output_real, lab_real)
        gw_real = torch.autograd.grad(loss_real, net_parameters)
        gw_real = list((_.detach().clone() for _ in gw_real))

        output_syn = model(img_syn)
        loss_syn = criterion(output_syn, lab_syn)
        gw_syn = torch.autograd.grad(loss_syn, net_parameters, create_graph=True)
        loss += match_mse_loss(gw_syn, gw_real)

      total_loss += loss
      optimizer_img.zero_grad()
      loss.backward()
      optimizer_img.step()

      image_syn_train, label_syn_train = copy.deepcopy(syn_image.detach()), copy.deepcopy(syn_lable.detach())  # avoid any unaware modification
      train_model(model, image_syn_train, label_syn_train, NUMBER_STEP_OPT)

  random = "Gaussian noise" if randmon_init else "real training images"
  save_name = os.path.join(SAVEPATH, 'vis_%s_%s_%s_%d.png'%(model_name, dataset_name, random, IMAGE_PER_CLASS))
  image_syn_vis = copy.deepcopy(syn_image.detach().cpu())
  save_image(image_syn_vis, save_name, nrow=IMAGE_PER_CLASS)
  make_grid(image_syn_vis, nrow=IMAGE_PER_CLASS)
  net_eval = get_network(eval_mode_name, channel, num_classes, im_size).to(DEVICE)
  image_syn_eval = copy.deepcopy(syn_image.detach())

  print('%s training begins'%get_time())
  acc_test = train_and_eval(net_eval, image_syn_eval, syn_lable, dst_test)
  print('%s training end'%get_time())
  return acc_test

In [18]:
DATAPATH = "/content/drive/MyDrive/MNIST"
SAVEPATH = "/content/drive/MyDrive/MNIST_condense"

In [ ]:
acc_list_MNIST = condensed_images_and_evaluate("MNIST", "ConvNet", "ConvNet")

In [16]:
BATCH_SIZE = 128
net_width, net_depth, net_act, net_norm, net_pooling = get_default_convnet_setting()
model_convnet7 = ConvNet7(channel=3, num_classes=2, net_width=net_width, net_depth=7, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling, im_size=(224,224)).to(DEVICE)

In [15]:
#################
#https://github.com/caiquanshi/ECE1512_2022F_ProjectRepo_CaiquanShi_JunchengMo/blob/main/Project%20B/task1/projectb_task1.ipynb
#################
def condensed_images_and_evaluate_MHIST(dataset_name, model_name, eval_mode_name, randmon_init=False):
  channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset("MNIST", "/content/drive/MyDrive/MNIST")
  channel = 3
  num_classes = 2
  dst_train = train_set
  dst_test = test_set
  im_size = (224,224)

  class_index = {}
  data_save = []
  for i in range(len(dst_train)):
    lable = dst_train[i][1]
    if lable in class_index:
      class_index[lable].append(i)
    else:
      class_index[lable] = [i]
  images_all = [torch.unsqueeze(dst_train[i][0], dim=0) for i in range(len(dst_train))]
  images_all = torch.cat(images_all, dim=0).to(DEVICE)
  syn_image = []
  for c in range(num_classes):
    idx_shuffle = np.random.permutation(class_index[c])[:IMAGE_PER_CLASS]
    syn_image.append(images_all[idx_shuffle])
  syn_image = torch.cat(syn_image, dim=0).to(DEVICE)
  syn_image.requires_grad = True
  syn_lable = []
  for c in range(num_classes):
    syn_lable += [c for i in range(IMAGE_PER_CLASS)]
  if randmon_init:
    syn_image = torch.randn(size=(num_classes*IMAGE_PER_CLASS, channel, im_size[0], im_size[1]), dtype=torch.float, requires_grad=True, device=DEVICE)
  syn_lable = torch.tensor(syn_lable,device=DEVICE)
  optimizer_img = torch.optim.SGD([syn_image, ], lr=LR_IMG)
  criterion = nn.CrossEntropyLoss().to(DEVICE)
  total_loss = 0
  print('%s training begins'%get_time())
  for exp in range(NUMBER_INITIALIZATIONS):
    #model = get_network(model_name, channel, num_classes, im_size).to(DEVICE)
    model = ConvNet7(channel=3, num_classes=2, net_width=net_width, net_depth=7, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling, im_size=(224,224)).to(DEVICE)
    model.train()
    net_parameters = list(model.parameters())

    for it in range(ITERATIONS):
      loss = torch.tensor(0.0).to(DEVICE)
      for c in range(num_classes):
        strat_index = c*IMAGE_PER_CLASS
        img_real = images_all[np.random.permutation(class_index[c])[:BATCH_SIZE]]
        lab_real = torch.ones((BATCH_SIZE,), device=DEVICE, dtype=torch.long) * c
        img_syn = syn_image[strat_index: strat_index + IMAGE_PER_CLASS].reshape((IMAGE_PER_CLASS, channel, im_size[0], im_size[1]))
        lab_syn = torch.ones((IMAGE_PER_CLASS,), device=DEVICE, dtype=torch.long) * c
        output_real = model(img_real)
        loss_real = criterion(output_real, lab_real)
        gw_real = torch.autograd.grad(loss_real, net_parameters)
        gw_real = list((_.detach().clone() for _ in gw_real))

        output_syn = model(img_syn)
        loss_syn = criterion(output_syn, lab_syn)
        gw_syn = torch.autograd.grad(loss_syn, net_parameters, create_graph=True)
        loss += match_mse_loss(gw_syn, gw_real)

      total_loss += loss
      optimizer_img.zero_grad()
      loss.backward()
      optimizer_img.step()

      image_syn_train, label_syn_train = copy.deepcopy(syn_image.detach()), copy.deepcopy(syn_lable.detach())  # avoid any unaware modification
      train_model(model, image_syn_train, label_syn_train, NUMBER_STEP_OPT)

  random = "Gaussian noise" if randmon_init else "real training images"
  save_name = os.path.join(SAVEPATH, 'vis_%s_%s_%s_%d.png'%(model_name, dataset_name, random, IMAGE_PER_CLASS))
  image_syn_vis = copy.deepcopy(syn_image.detach().cpu())
  save_image(image_syn_vis, save_name, nrow=IMAGE_PER_CLASS)
  make_grid(image_syn_vis, nrow=IMAGE_PER_CLASS)
  #net_eval = get_network(eval_mode_name, channel, num_classes, im_size).to(DEVICE)
  net_eval = ConvNet7(channel=3, num_classes=2, net_width=net_width, net_depth=7, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling, im_size=(224,224)).to(DEVICE)
  image_syn_eval = copy.deepcopy(syn_image.detach())

  print('%s training begins'%get_time())
  acc_test = train_and_eval(net_eval, image_syn_eval, syn_lable, dst_test)
  print('%s training end'%get_time())
  return acc_test

In [ ]:
acc_list_MHIST = condensed_images_and_evaluate_MHIST("MHIST", "ConvNet7", "ConvNet7")

[2023-12-06 03:53:12] training begins
[2023-12-06 04:34:08] training begins
Epoch: 0 	Training Cost: 8.991707	 Valid Acc: 0.3222222328186035.
Epoch: 1 	Training Cost: 6.269272	 Valid Acc: 0.561904788017273.
Epoch: 2 	Training Cost: 4.919905	 Valid Acc: 0.3365079462528229.
Epoch: 3 	Training Cost: 3.656777	 Valid Acc: 0.47460317611694336.
Epoch: 4 	Training Cost: 2.551062	 Valid Acc: 0.47460317611694336.
Epoch: 5 	Training Cost: 1.760011	 Valid Acc: 0.5444444417953491.
Epoch: 6 	Training Cost: 1.343847	 Valid Acc: 0.45079365372657776.
Epoch: 7 	Training Cost: 0.994573	 Valid Acc: 0.4650793671607971.
Epoch: 8 	Training Cost: 0.767305	 Valid Acc: 0.45079365372657776.
Epoch: 9 	Training Cost: 0.614199	 Valid Acc: 0.4365079402923584.
Epoch: 10 	Training Cost: 0.514436	 Valid Acc: 0.4396825432777405.
Epoch: 11 	Training Cost: 0.439834	 Valid Acc: 0.44285714626312256.
Epoch: 12 	Training Cost: 0.385893	 Valid Acc: 0.43809524178504944.
Epoch: 13 	Training Cost: 0.338105	 Valid Acc: 0.438095241

#2.d

In [19]:
acc_list_MNIST_Gaussian = condensed_images_and_evaluate("MNIST", "ConvNet", "ConvNet", True)

[2023-12-06 14:26:22] training begins
[2023-12-06 15:59:52] training begins
Epoch: 0 	Training Cost: 168.761447	 Valid Acc: 0.10159999877214432.
Epoch: 1 	Training Cost: 130.074834	 Valid Acc: 0.2402999997138977.
Epoch: 2 	Training Cost: 126.449328	 Valid Acc: 0.4551999866962433.
Epoch: 3 	Training Cost: 119.921593	 Valid Acc: 0.3172999918460846.
Epoch: 4 	Training Cost: 119.272793	 Valid Acc: 0.439300000667572.
Epoch: 5 	Training Cost: 110.628913	 Valid Acc: 0.1559000015258789.
Epoch: 6 	Training Cost: 107.264309	 Valid Acc: 0.3506999909877777.
Epoch: 7 	Training Cost: 108.479375	 Valid Acc: 0.4382999837398529.
Epoch: 8 	Training Cost: 101.438738	 Valid Acc: 0.33219999074935913.
Epoch: 9 	Training Cost: 100.466031	 Valid Acc: 0.30059999227523804.
Epoch: 10 	Training Cost: 95.636315	 Valid Acc: 0.47759997844696045.
Epoch: 11 	Training Cost: 94.670391	 Valid Acc: 0.5309999585151672.
Epoch: 12 	Training Cost: 95.813981	 Valid Acc: 0.44259998202323914.
Epoch: 13 	Training Cost: 91.725396	

In [21]:
BATCH_SIZE = 128
acc_list_MHIST_Gaussian = condensed_images_and_evaluate_MHIST("MHIST", "ConvNet7", "ConvNet7", True)

[2023-12-06 16:18:10] training begins
[2023-12-06 18:32:36] training begins
Epoch: 0 	Training Cost: 7.842100	 Valid Acc: 0.6333333253860474.
Epoch: 1 	Training Cost: 5.361252	 Valid Acc: 0.5317460298538208.
Epoch: 2 	Training Cost: 2.949223	 Valid Acc: 0.5365079641342163.
Epoch: 3 	Training Cost: 1.523403	 Valid Acc: 0.5523809790611267.
Epoch: 4 	Training Cost: 0.940079	 Valid Acc: 0.5428571701049805.
Epoch: 5 	Training Cost: 0.673972	 Valid Acc: 0.5285714268684387.
Epoch: 6 	Training Cost: 0.523605	 Valid Acc: 0.5206349492073059.
Epoch: 7 	Training Cost: 0.427342	 Valid Acc: 0.5047619342803955.
Epoch: 8 	Training Cost: 0.361695	 Valid Acc: 0.5.
Epoch: 9 	Training Cost: 0.310622	 Valid Acc: 0.4888888895511627.
Epoch: 10 	Training Cost: 0.272902	 Valid Acc: 0.5.
Epoch: 11 	Training Cost: 0.242280	 Valid Acc: 0.49365079402923584.
Epoch: 12 	Training Cost: 0.216573	 Valid Acc: 0.5031746029853821.
Epoch: 13 	Training Cost: 0.195417	 Valid Acc: 0.49365079402923584.
Epoch: 14 	Training Cost

#3. Cross-architecture Generalization.

In [25]:
condensed_images_and_evaluate("MNIST", "ConvNet", "AlexNet")

[2023-12-06 22:53:15] training begins
[2023-12-07 00:31:52] training begins
Epoch: 0 	Training Cost: 115.587542	 Valid Acc: 0.11659999936819077.
Epoch: 1 	Training Cost: 113.451244	 Valid Acc: 0.0982000008225441.
Epoch: 2 	Training Cost: 111.990371	 Valid Acc: 0.10819999873638153.
Epoch: 3 	Training Cost: 109.741653	 Valid Acc: 0.18140000104904175.
Epoch: 4 	Training Cost: 106.350724	 Valid Acc: 0.10089999437332153.
Epoch: 5 	Training Cost: 103.251298	 Valid Acc: 0.621999979019165.
Epoch: 6 	Training Cost: 96.646477	 Valid Acc: 0.5633000135421753.
Epoch: 7 	Training Cost: 91.891493	 Valid Acc: 0.44119998812675476.
Epoch: 8 	Training Cost: 88.682129	 Valid Acc: 0.6237999796867371.
Epoch: 9 	Training Cost: 84.548551	 Valid Acc: 0.5566999912261963.
Epoch: 10 	Training Cost: 78.374485	 Valid Acc: 0.5960999727249146.
Epoch: 11 	Training Cost: 75.461601	 Valid Acc: 0.5995999574661255.
Epoch: 12 	Training Cost: 74.107934	 Valid Acc: 0.5949999690055847.
Epoch: 13 	Training Cost: 71.147687	 Val

[tensor(0.1166, device='cuda:0'),
 tensor(0.0982, device='cuda:0'),
 tensor(0.1082, device='cuda:0'),
 tensor(0.1814, device='cuda:0'),
 tensor(0.1009, device='cuda:0'),
 tensor(0.6220, device='cuda:0'),
 tensor(0.5633, device='cuda:0'),
 tensor(0.4412, device='cuda:0'),
 tensor(0.6238, device='cuda:0'),
 tensor(0.5567, device='cuda:0'),
 tensor(0.5961, device='cuda:0'),
 tensor(0.5996, device='cuda:0'),
 tensor(0.5950, device='cuda:0'),
 tensor(0.5950, device='cuda:0'),
 tensor(0.5774, device='cuda:0'),
 tensor(0.4670, device='cuda:0'),
 tensor(0.5126, device='cuda:0'),
 tensor(0.5741, device='cuda:0'),
 tensor(0.6321, device='cuda:0'),
 tensor(0.6016, device='cuda:0')]

In [22]:
condensed_images_and_evaluate("MNIST", "ConvNet", "VGG11")

[2023-12-06 18:36:14] training begins
[2023-12-06 20:13:48] training begins
Epoch: 0 	Training Cost: 268.037476	 Valid Acc: 0.0973999947309494.
Epoch: 1 	Training Cost: 161.781989	 Valid Acc: 0.0973999947309494.
Epoch: 2 	Training Cost: 144.536239	 Valid Acc: 0.0957999974489212.
Epoch: 3 	Training Cost: 146.139456	 Valid Acc: 0.0957999974489212.
Epoch: 4 	Training Cost: 138.595403	 Valid Acc: 0.09799999743700027.
Epoch: 5 	Training Cost: 131.886279	 Valid Acc: 0.17489999532699585.
Epoch: 6 	Training Cost: 129.902572	 Valid Acc: 0.33149999380111694.
Epoch: 7 	Training Cost: 117.027888	 Valid Acc: 0.28189998865127563.
Epoch: 8 	Training Cost: 113.296620	 Valid Acc: 0.26829999685287476.
Epoch: 9 	Training Cost: 113.713710	 Valid Acc: 0.16599999368190765.
Epoch: 10 	Training Cost: 106.555901	 Valid Acc: 0.3779999911785126.
Epoch: 11 	Training Cost: 102.909308	 Valid Acc: 0.28290000557899475.
Epoch: 12 	Training Cost: 102.609542	 Valid Acc: 0.4041000008583069.
Epoch: 13 	Training Cost: 97.3

[tensor(0.0974, device='cuda:0'),
 tensor(0.0974, device='cuda:0'),
 tensor(0.0958, device='cuda:0'),
 tensor(0.0958, device='cuda:0'),
 tensor(0.0980, device='cuda:0'),
 tensor(0.1749, device='cuda:0'),
 tensor(0.3315, device='cuda:0'),
 tensor(0.2819, device='cuda:0'),
 tensor(0.2683, device='cuda:0'),
 tensor(0.1660, device='cuda:0'),
 tensor(0.3780, device='cuda:0'),
 tensor(0.2829, device='cuda:0'),
 tensor(0.4041, device='cuda:0'),
 tensor(0.3939, device='cuda:0'),
 tensor(0.4410, device='cuda:0'),
 tensor(0.4625, device='cuda:0'),
 tensor(0.3689, device='cuda:0'),
 tensor(0.4716, device='cuda:0'),
 tensor(0.4796, device='cuda:0'),
 tensor(0.4531, device='cuda:0')]

In [23]:
def condensed_images_and_evaluate_MHIST_2(dataset_name, model_name, eval_mode_name, randmon_init=False):
  channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset("MNIST", "/content/drive/MyDrive/MNIST")
  channel = 3
  num_classes = 2
  dst_train = train_set
  dst_test = test_set
  im_size = (224,224)

  class_index = {}
  data_save = []
  for i in range(len(dst_train)):
    lable = dst_train[i][1]
    if lable in class_index:
      class_index[lable].append(i)
    else:
      class_index[lable] = [i]
  images_all = [torch.unsqueeze(dst_train[i][0], dim=0) for i in range(len(dst_train))]
  images_all = torch.cat(images_all, dim=0).to(DEVICE)
  syn_image = []
  for c in range(num_classes):
    idx_shuffle = np.random.permutation(class_index[c])[:IMAGE_PER_CLASS]
    syn_image.append(images_all[idx_shuffle])
  syn_image = torch.cat(syn_image, dim=0).to(DEVICE)
  syn_image.requires_grad = True
  syn_lable = []
  for c in range(num_classes):
    syn_lable += [c for i in range(IMAGE_PER_CLASS)]
  if randmon_init:
    syn_image = torch.randn(size=(num_classes*IMAGE_PER_CLASS, channel, im_size[0], im_size[1]), dtype=torch.float, requires_grad=True, device=DEVICE)
  syn_lable = torch.tensor(syn_lable,device=DEVICE)
  optimizer_img = torch.optim.SGD([syn_image, ], lr=LR_IMG)
  criterion = nn.CrossEntropyLoss().to(DEVICE)
  total_loss = 0
  print('%s training begins'%get_time())
  for exp in range(NUMBER_INITIALIZATIONS):
    #model = get_network(model_name, channel, num_classes, im_size).to(DEVICE)
    model = ConvNet7(channel=3, num_classes=2, net_width=net_width, net_depth=7, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling, im_size=(224,224)).to(DEVICE)
    model.train()
    net_parameters = list(model.parameters())

    for it in range(ITERATIONS):
      loss = torch.tensor(0.0).to(DEVICE)
      for c in range(num_classes):
        strat_index = c*IMAGE_PER_CLASS
        img_real = images_all[np.random.permutation(class_index[c])[:BATCH_SIZE]]
        lab_real = torch.ones((BATCH_SIZE,), device=DEVICE, dtype=torch.long) * c
        img_syn = syn_image[strat_index: strat_index + IMAGE_PER_CLASS].reshape((IMAGE_PER_CLASS, channel, im_size[0], im_size[1]))
        lab_syn = torch.ones((IMAGE_PER_CLASS,), device=DEVICE, dtype=torch.long) * c
        output_real = model(img_real)
        loss_real = criterion(output_real, lab_real)
        gw_real = torch.autograd.grad(loss_real, net_parameters)
        gw_real = list((_.detach().clone() for _ in gw_real))

        output_syn = model(img_syn)
        loss_syn = criterion(output_syn, lab_syn)
        gw_syn = torch.autograd.grad(loss_syn, net_parameters, create_graph=True)
        loss += match_mse_loss(gw_syn, gw_real)

      total_loss += loss
      optimizer_img.zero_grad()
      loss.backward()
      optimizer_img.step()

      image_syn_train, label_syn_train = copy.deepcopy(syn_image.detach()), copy.deepcopy(syn_lable.detach())  # avoid any unaware modification
      train_model(model, image_syn_train, label_syn_train, NUMBER_STEP_OPT)

  random = "Gaussian noise" if randmon_init else "real training images"
  save_name = os.path.join(SAVEPATH, 'vis_%s_%s_%s_%d.png'%(model_name, dataset_name, random, IMAGE_PER_CLASS))
  image_syn_vis = copy.deepcopy(syn_image.detach().cpu())
  save_image(image_syn_vis, save_name, nrow=IMAGE_PER_CLASS)
  make_grid(image_syn_vis, nrow=IMAGE_PER_CLASS)
  net_eval = get_network(eval_mode_name, channel, num_classes, im_size).to(DEVICE)
  #net_eval = ConvNet7(channel=3, num_classes=2, net_width=net_width, net_depth=7, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling, im_size=(224,224)).to(DEVICE)
  image_syn_eval = copy.deepcopy(syn_image.detach())

  print('%s training begins'%get_time())
  acc_test = train_and_eval(net_eval, image_syn_eval, syn_lable, dst_test)
  print('%s training end'%get_time())
  return acc_test

In [ ]:
acc_list_MHIST_cross = condensed_images_and_evaluate_MHIST_2("MHIST", "ConvNet", "VGG11")

In [ ]:
acc_list_MHIST_cross = condensed_images_and_evaluate_MHIST_2("MHIST", "ConvNet7", "AlexNet")